In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout
from keras.models import Model, save, load_model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam


from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob

In [2]:
Trainpath = "/content/drive/MyDrive/IEEE_Project/Dataset/Training/"

Validpath = "/content/drive/MyDrive/IEEE_Project/Dataset/Validation/"

datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range = 40, 
                    width_shift_range = 0.2, 
                    height_shift_range = 0.2, 
                    shear_range = 0.2)

train_generator = datagen.flow_from_directory(
    Trainpath,
    target_size=(224,224),
    batch_size = 32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    Validpath,
    target_size=(224,224),
    batch_size = 32,
    class_mode='categorical'
)

Found 300 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [3]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [5]:
# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

x = Dense(1024, activation='relu')(x)

x = Dense(512, activation='relu')(x)

x = Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = Dense(3, activation='softmax')(x)

model = Model(base_model.input, x)

# model.compile(optimizer = RMSprop(lr=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

model.compile(optimizer = Adam(lr=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
model.fit_generator(train_generator, validation_data = validation_generator, epochs = 25)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/25
10/10 [==============================] - 105s 10s/step - loss: 0.8570 - acc: 0.6100 - val_loss: 0.7133 - val_acc: 0.7133
Epoch 2/25
10/10 [==============================] - 6s 610ms/step - loss: 0.2600 - acc: 0.9333 - val_loss: 0.6144 - val_acc: 0.7867
Epoch 3/25
10/10 [==============================] - 6s 655ms/step - loss: 0.1094 - acc: 0.9767 - val_loss: 0.5341 - val_acc: 0.8200
Epoch 4/25
10/10 [==============================] - 6s 605ms/step - loss: 0.0865 - acc: 0.9667 - val_loss: 0.3273 - val_acc: 0.8667
Epoch 5/25
10/10 [==============================] - 6s 644ms/step - loss: 0.0377 - acc: 0.9967 - val_loss: 0.2563 - val_acc: 0.8800
Epoch 6/25
10/10 [==============================] - 6s 609ms/step - loss: 0.0145 - acc: 0.9967 - val_loss: 0.4295 - val_acc: 0.8467
Epoch 7/25
10/10 [==============================] - 6s 605ms/step - loss: 0.0297 - acc: 0.9933 - val_loss: 0.3593 - val_acc: 0.8600
Epoch 8/25
10/10 [==============================] - 6s 651ms/step - loss: 0.

In [7]:
model.save("/content/drive/MyDrive/IEEE_Project/gesture_model.h5")